In [1]:
!git clone https://github.com/susannapaoli/multimodal-memes.git

Cloning into 'multimodal-memes'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 144 (delta 51), reused 94 (delta 27), pack-reused 0
Receiving objects: 100% (144/144), 3.73 MiB | 27.31 MiB/s, done.
Resolving deltas: 100% (51/51), done.


**Get data from Kaggle**

In [2]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
files.upload()

Mounted at /content/drive


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"susannapaoli","key":"990ea74608e878a29ea3a7599a71e25f"}'}

In [3]:
!pip install -q kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!pwd

/content


In [7]:
!kaggle datasets download -d parthplc/facebook-hateful-meme-dataset

100% 3.35G/3.35G [01:29<00:00, 43.0MB/s]
100% 3.35G/3.35G [01:30<00:00, 40.0MB/s]


In [ ]:
!unzip /content/facebook-hateful-meme-dataset.zip

In [9]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.8 MB/s eta 0:00:00


In [10]:
'''
IMPORTING NECESSARY MODULES
'''

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('/content/multimodal-memes/BERT')
sys.path.append('/content/multimodal-memes')
sys.path.append('/content/multimodal-memes/Utils')
sys.path.append('/content/multimodal-memes/Data')
sys.path.append('/content/multimodal-memes/Resnet')
sys.path.append('/content/multimodal-memes/Data/data/img')

from dataloader import mydataset
#from Load_model import load
from plot_curves import plot_loss, plot_acc
from trainer_resnet_BERT import train, test_classify

'''
For ResNet/ ResNeXt
'''
from resnet_model import ResNet,Bottleneck, resnext101_32x8d


'''
For BERT
'''
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

**Device**

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


**Dataloading Scheme**

In [12]:
%cd /content/data

/content/data


In [13]:
trainlist = 'train.jsonl'
validlist = 'dev.jsonl'

In [14]:
'''
Train Dataloader
''' 
train_dataset = mydataset(trainlist, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 16, num_workers=16,pin_memory=True)


'''
Validation Dataloader
''' 
validation_dataset = mydataset(validlist, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 8, num_workers=16,pin_memory=True)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg

In [15]:
train_dataset[2000]

(tensor([[[ 0.2111,  0.2282,  0.2453,  ...,  0.3994,  0.3823,  0.3994],
          [ 0.2453,  0.2624,  0.2624,  ...,  0.4166,  0.4166,  0.3994],
          [ 0.2624,  0.2624,  0.2624,  ...,  0.4337,  0.4337,  0.3994],
          ...,
          [ 0.0056,  0.0227,  0.0056,  ..., -1.6042, -1.7240, -1.9809],
          [ 0.0056, -0.0116, -0.0116,  ..., -1.9295, -1.9467, -1.9295],
          [-0.0116, -0.0116, -0.0116,  ..., -1.9295, -1.9467, -1.9467]],
 
         [[ 0.0826,  0.1001,  0.1001,  ...,  0.2052,  0.1877,  0.1877],
          [ 0.1176,  0.1176,  0.1176,  ...,  0.2227,  0.1877,  0.1702],
          [ 0.1176,  0.1352,  0.1176,  ...,  0.2402,  0.1877,  0.1877],
          ...,
          [-0.1099, -0.0924, -0.0924,  ..., -1.4930, -1.6155, -1.8606],
          [-0.1275, -0.1099, -0.1099,  ..., -1.7731, -1.7906, -1.7731],
          [-0.1275, -0.1099, -0.1099,  ..., -1.7731, -1.7906, -1.7906]],
 
         [[ 0.0431,  0.0605,  0.0605,  ..., -0.0964, -0.0790, -0.0615],
          [ 0.0779,  0.0779,

**Model Definition**

In [ ]:
'''
Model1 ResNeXt101_32x8d
'''
# Image_model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)
Image_model = resnext101_32x8d()

Image_model.fc = nn.Sequential(
    nn.Linear(Image_model.fc.in_features, 2)
    )

Image_model = nn.DataParallel(Image_model).to(device)

Image_model.to(device)

In [17]:
'''
Model 2 BERT

Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 

Text_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,   
    output_attentions = False, 
    output_hidden_states = True
)

Text_model = nn.DataParallel(Text_model).to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
'''
Fusion
'''
class FusionNet(nn.Module):
    
    def __init__(self, num_classes, drop_prob = 0.1):
        super(FusionNet, self).__init__()
        
        self.pooler = nn.Linear(in_features=768, out_features=768)
        
        self.concat = nn.Linear(in_features=768+2048, out_features= 512)
        
        self.bn = nn.BatchNorm1d(512)
        self.bn1 = nn.BatchNorm1d(768)
        self.bn2 = nn.BatchNorm1d(2048)
    
        self.dropout = nn.Dropout(drop_prob)
        
        self.classify = nn.Linear(in_features = 512, out_features = num_classes)
        
        
    def forward(self, text_features, image_features):
#         x = torch.tanh(self.pooler(text_features))
#         x = self.dropout(x)

        text_features = self.bn1(text_features)
        image_features = self.bn2(image_features)
      
        fused =  torch.cat((text_features, image_features), dim=1)
        
        x = self.concat(fused)
        x = F.relu(self.bn(x))        
        
        x = F.relu(self.classify(x)) 

        return x



In [19]:
Fusion_model = FusionNet(num_classes =2 , drop_prob = 0.1)
Fusion_model = nn.DataParallel(Fusion_model).to(device)

**Hyperparameters**

In [22]:
'''
Loss Function
'''
criterion = nn.CrossEntropyLoss()


'''
Optimizer
'''
optimizer = torch.optim.SGD(Fusion_model.parameters(), lr=0.01, weight_decay=1e-4, momentum=0.9)

'''
Number of training epochs.
'''
num_Epochs = 1

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 4, gamma = 0.1)

In [21]:
model_name = 'Concatenation'
model_path = './saved_model_checkpoints/'+model_name

In [ ]:
writer = SummaryWriter(model_name)

train(Image_model, Text_model, Fusion_model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, model_path, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

**Evaluate on Validation set**

In [ ]:
test_classify(Fusion_model, validation_dataloader, criterion, device)